In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from tqdm import tqdm
import math
import random
import numpy as np
import pandas as pd
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


device = "cuda" if torch.cuda.is_available() else "cpu"

class FeatureRegression(nn.Module):
    def __init__(self, input_size):
        super(FeatureRegression, self).__init__()
        self.build(input_size)

    def build(self, input_size):
        self.W = Parameter(torch.Tensor(input_size, input_size))
        self.b = Parameter(torch.Tensor(input_size))

        m = torch.ones(input_size, input_size).cuda() - torch.eye(input_size, input_size).cuda()
        self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, x):
        z_h = F.linear(x, self.W * Variable(self.m), self.b)
        return z_h

class TemporalDecay(nn.Module):
    def __init__(self, input_size, output_size, diag = False):
        super(TemporalDecay, self).__init__()
        self.diag = diag

        self.build(input_size, output_size)

    def build(self, input_size, output_size):
        self.W = Parameter(torch.Tensor(output_size, input_size)).cuda()
        self.b = Parameter(torch.Tensor(output_size)).cuda()
        self.relu = nn.ReLU(inplace=False)
        if self.diag == True:
            assert(input_size == output_size)
            m = torch.eye(input_size, input_size).cuda()
            self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, d):
        gamma = self.relu(F.linear(d, self.W, self.b))
        gamma = torch.exp(-gamma)
        return gamma

# Generator 모델
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Generator, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.batch_norm = nn.BatchNorm1d(hidden_size)

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)
        self.hidden_dropout = nn.Dropout(p=0.5)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        rbfs = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]

        hid = torch.zeros((rbfs.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(rbfs.size(1)):

            r = rbfs[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]

            gamma_r = self.temp_decay_r(d)
            gamma_h = self.temp_decay_h(d)

            hid = hid * gamma_h
            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(r - x_hat) * m) / (torch.sum(m) + 1e-5)

            r_c = m * r + (1 - m) * x_hat

            r_hat = self.z_layer(r_c)

            x_loss += torch.sum(torch.abs(r - r_hat) * m) / (torch.sum(m) + 1e-5)

            beta_weight = torch.cat([gamma_r, m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * r_hat + (1 - beta) * x_hat
            x_loss += torch.sum(torch.abs(r - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * r + (1 - m) * c_hat

            gru_input = torch.cat([c_c, m], dim = 1)
            c_hat_list.append(c_hat.unsqueeze(1))
            imputations.append(c_c.unsqueeze(1))
            
            # GRU cell
            self.hidden_dropout(hid)
            hid = self.grucell(gru_input, hid)
            
        imputations = torch.cat(imputations, dim = 1)
        c_hat_list = torch.cat(c_hat_list, dim = 1)

        return  c_hat_list, imputations, x_loss / rbfs.size(1)

In [8]:
from torch.utils.data import Dataset, DataLoader
class MyDataset(Dataset):
    def __init__(self, dataset, q):
        self.data = dataset
        self.q = q

    def __len__(self):
        return self.data.shape[1] // self.q

    def __getitem__(self, index):
        return self.data[:,index * self.q : index * self.q + self.q,:]

def make_deltas(masks):
    deltas = []
    for h in range(len(masks)):
        if h == 0:
            deltas.append([1 for _ in range(masks.shape[1])])
        else:
            deltas.append([1 for _ in range(masks.shape[1])] + (1-masks[h]) * deltas[-1])
    
    return list(deltas)

def binary_sampler(p, rows, cols):
    unif_random_matrix = np.random.uniform(0., 1., size = [rows, cols])
    binary_random_matrix = 1. * (unif_random_matrix < p)

    return binary_random_matrix

def hint_matrix(B, Masks):
    Hint = B * Masks + 0.5 * (1-B)

    Hint = torch.from_numpy(Hint).to(torch.float32)
    return Hint
    
def missing_data_rbf(df,rbf, batch_size, p):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    binary_random_matrix = binary_sampler(p, shp[0], shp[1])

    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)
    Hint = hint_matrix(binary_random_matrix, masks)
    
    deltas = np.array(make_deltas(masks))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([rbf_x.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), Hint.unsqueeze_(0)], dim = 0)

    mydata  = MyDataset(dataset, batch_size)
    data = DataLoader(mydata, batch_size, shuffle=True)

    return data

In [9]:
def train_G(dataset, Generator, lr, epochs):
    imputation_list = []

    optimizer_G = optim.Adam(Generator.parameters(), lr=lr)

    progress = tqdm(range(epochs))
    Generator.to(device)

    for epoch in progress:
        batch_loss_G = 0.0
        batch_loss_D = 0.0

        for data in dataset:

            data = data.to(device)
            hint = data[:,3,::].clone().detach()
            Mask = data[:,2,::].clone().detach()

            c_hat_list, imputations, x_loss = Generator(data)
            optimizer_G.zero_grad()
            x_loss.backward(retain_graph=True)
            optimizer_G.step()

            imputation_list.append(imputations)
            batch_loss_G += x_loss

        progress.set_description("G_loss: {}".format(batch_loss_G / len(dataset)))
    
    return imputation_list

In [10]:
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = "air"
batch_size = 64
dataset = missing_data_rbf(df, rbfpath, batch_size, 0.9)

In [12]:
rbf_df = pd.read_csv("./RBFresult/air_20_8.0_drop_0.2.csv")

In [15]:
(df - df.mean()) / df.std()

,001001,001002,001003,001004,001005,001006,001007,001008,001009,001010,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
0,0.729387,0.202664,0.305591,0.150003,0.135624,0.057313,0.094235,0.024987,0.039286,-0.075675,...,NaN,0.231177,0.756275,NaN,0.580793,0.415816,0.021781,-0.129339,NaN,NaN
1,0.546073,0.147144,0.525986,0.264572,0.045654,0.216440,0.333405,0.160917,0.248561,0.006787,...,NaN,0.129904,0.633067,NaN,0.580793,0.367683,0.177156,-0.059496,NaN,NaN
2,0.585355,0.188784,0.649958,0.366411,0.056900,0.349046,0.460024,0.235061,0.366278,0.183491,...,NaN,0.317983,0.571463,NaN,0.679165,0.351639,0.425754,0.050257,NaN,NaN
3,0.611542,0.355345,0.746381,0.468250,0.124378,0.468391,0.516299,0.247418,0.431677,0.207052,...,NaN,0.375853,0.571463,NaN,0.744746,0.431860,0.472367,0.130077,NaN,NaN
4,NaN,0.480267,0.842804,0.544629,0.203102,0.547955,0.685125,0.333919,0.523234,0.242393,...,NaN,0.173307,0.556062,NaN,0.810327,0.383727,0.550054,0.179965,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,0.048507,-0.033298,-0.176523,-0.180974,-0.269242,-0.260941,-0.229347,-0.246873,-0.130750,-0.323061,...,NaN,NaN,0.171037,0.009604,0.203702,1.025498,0.332530,-0.398732,-0.429225,NaN
8755,0.061601,-0.019418,-0.135199,-0.130055,-0.156780,-0.260941,-0.285622,-0.333374,-0.287707,-0.299501,...,NaN,NaN,-0.059977,-0.018071,0.220097,1.009454,0.860802,-0.039541,-0.377575,NaN
8756,-0.030056,-0.005538,-0.038776,-0.206434,-0.145533,-0.181378,-0.130865,-0.172729,-0.091511,-0.240599,...,NaN,NaN,0.063231,0.023441,0.334864,1.025498,1.031714,0.589044,-0.284604,NaN
8757,NaN,NaN,0.057646,NaN,NaN,-0.088554,-0.032384,NaN,-0.026113,NaN,...,NaN,NaN,NaN,0.078791,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
(rbf_df - rbf_df.mean()) / rbf_df.std()

,001001,001002,001003,001004,001005,001006,001007,001008,001009,001010,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
0,0.833702,0.074850,0.347782,0.198880,0.178503,0.126357,0.182780,0.115592,0.094246,-0.043076,...,-0.053620,0.222963,0.748346,-0.311618,0.216050,0.458980,0.099035,-0.035186,-0.343594,-0.420327
1,0.884523,0.229706,0.530275,0.321159,0.199503,0.277548,0.320911,0.300777,0.255071,0.051002,...,0.137924,0.259383,0.751663,-0.233680,0.393282,0.513269,0.245915,0.040810,-0.204572,-0.305951
2,0.854783,0.307665,0.723294,0.406509,0.139408,0.383615,0.421295,0.514432,0.393861,0.173122,...,0.379266,0.376184,0.641818,-0.129991,0.522623,0.475514,0.332519,0.239367,-0.029407,-0.161838
3,0.797969,0.382020,0.837884,0.464174,0.124071,0.477686,0.530302,0.717413,0.529407,0.288889,...,0.672896,0.510132,0.564856,0.003552,0.674533,0.428275,0.443649,0.455692,0.183708,0.013497
4,0.680214,0.576192,0.800103,0.426261,0.231408,0.586270,0.669133,0.814170,0.649634,0.325653,...,1.017126,0.516252,0.569529,0.169826,0.824753,0.329915,0.607453,0.527583,0.433548,0.219047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,0.190555,-0.068424,-0.175941,-0.172720,-0.262101,-0.253210,-0.212252,-0.198784,-0.133938,-0.321118,...,0.178321,-0.296517,0.159499,-0.006098,0.162669,1.012385,0.570752,-0.349070,-0.310987,-0.580916
8755,0.145157,-0.140374,-0.166147,-0.169738,-0.211978,-0.234623,-0.205259,-0.168571,-0.112179,-0.292837,...,0.112672,-0.296055,0.109013,-0.013756,0.192919,1.060401,0.780759,-0.016766,-0.257222,-0.580590
8756,0.093035,-0.186031,-0.111841,-0.133577,-0.143286,-0.174276,-0.049471,-0.117937,-0.060297,-0.230917,...,0.055072,-0.295588,0.096265,0.042617,0.268784,1.174698,1.114924,0.572203,-0.168022,-0.580264
8757,0.043069,-0.138666,-0.043633,-0.115748,-0.132931,-0.144740,0.291237,-0.108921,-0.020099,-0.163647,...,0.003419,-0.295117,0.094204,0.133922,0.279773,1.406187,1.501001,1.218251,-0.055078,-0.579938


In [11]:
G = Generator(36, 64)
imputation_list = train_G(dataset, G,  1e-3, 300)

G_loss: 1.968428373336792:   1%|▏         | 4/300 [00:04<05:39,  1.15s/it] 


KeyboardInterrupt: 

In [11]:
def val_missing_data_rbf(df,rbf):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([rbf_x.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0)], dim = 0).unsqueeze_(0)

    return dataset

def eval_model(model, rbf, realpath, dfpath):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    dataset = val_missing_data_rbf(df, rbf)
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    dataset = dataset.to(device)

    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
    real_scaler = (real - df.mean()) / df.std()

    df_scaler = ((df-df.mean()) / df.std()).values
    masks = ~np.isnan(df_scaler)
    masks = torch.from_numpy(masks).to(torch.float32)
    
    eval_masks = ~np.isnan(real_scaler.values)
    eval_masks = torch.from_numpy(eval_masks).to(torch.float32)

    test_masks = eval_masks - masks
    real_scaler = torch.nan_to_num(torch.from_numpy(real_scaler.values).to(torch.float32))
    
    model.eval()
    c_hat_list, imputations, x_loss = model(dataset)

    Nonscale_imputataion = pd.DataFrame(imputations[0].cpu().detach() , columns= df.columns)
    Nonscale_imputataion = (Nonscale_imputataion * df.std()) + df.mean()
    rbf_df = pd.DataFrame(rbf_df.values, columns= df.columns)
    rbf_df = (rbf_df * df.std()) + df.mean()
    
    real = real.fillna(0)
    df = df.fillna(0)
    print("Scale MAE :", torch.sum(torch.abs(imputations[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    print("Scale MRE :", torch.sum(torch.abs(imputations[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    print("RBF MAE :", np.sum(np.abs((rbf_df - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("RBF MRE :", np.sum(np.abs((rbf_df - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    print('observation MAE :', np.sum(np.abs((Nonscale_imputataion - df).values * masks.cpu().numpy())) / np.sum(masks.cpu().numpy()))

    return Nonscale_imputataion

In [12]:
realpath = 'pm25_ground.txt'
real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)

In [13]:
eval_model(G, rbfpath, 'pm25_ground.txt', dfpath)

Scale MAE : tensor(0.2228)
Scale MRE : tensor(0.3217)
Original MAE : 17.954595193516763
Original MRE : 0.2522054173453041
RBF MAE : 26.244345089085357
RBF MRE : 0.36865024996703916
observation MAE : 8.382986230262004


,001001,001002,001003,001004,001005,001006,001007,001008,001009,001010,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
0,112.979632,94.558028,109.012972,105.292300,104.623389,98.806747,102.341168,99.229637,100.945096,96.546109,...,76.657814,78.509137,93.531377,74.435169,89.525562,74.103350,88.368280,109.708148,100.071964,118.165729
1,116.773533,97.222771,112.847540,109.052867,106.300672,101.935345,106.501526,101.818824,104.554178,98.539958,...,93.285501,80.232474,96.495995,89.972203,93.647284,76.142415,91.739778,111.762013,97.526683,109.480425
2,119.546584,98.990375,116.139787,113.003751,108.182353,105.169956,110.835437,105.005373,108.455424,101.542161,...,101.235739,81.931966,98.443932,93.121400,97.075322,78.074774,95.394895,114.699009,102.464654,114.007791
3,121.463774,99.790112,119.029576,117.346926,110.447253,108.663489,115.472482,109.059993,112.779486,105.811113,...,102.226006,83.787046,99.414389,92.917473,99.833949,80.079464,99.463593,118.629706,105.957574,115.321597
4,115.965042,99.871445,121.933568,122.414836,113.389459,112.696303,120.620953,114.270940,117.706829,111.515048,...,104.394425,86.126502,99.783051,93.858217,102.235314,82.504918,104.119098,123.537302,109.087697,114.799340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,85.517552,66.028307,70.094838,65.534252,73.493956,66.055605,64.616278,64.528036,70.011654,69.110241,...,101.756341,110.175764,84.389473,76.380683,86.552459,119.705787,111.664071,96.144254,76.555851,65.435776
8755,82.107537,66.425545,73.556263,67.037746,74.498008,65.691271,72.138699,64.140210,70.108171,70.659920,...,109.730342,119.738275,77.498082,74.398222,80.346946,129.726792,125.189406,125.314947,75.624343,66.337765
8756,81.051560,69.216705,79.531669,70.265830,76.753433,67.533268,83.520990,65.965787,72.694449,74.851661,...,121.803290,132.171372,74.110290,75.558825,78.026563,139.256283,140.241324,160.064574,77.533807,70.336446
8757,84.923013,66.641258,85.776118,77.325712,84.580529,70.316736,95.181907,87.418769,76.202058,96.031915,...,129.920518,133.791729,79.738818,78.028144,66.076587,114.766354,139.852257,134.533681,98.645685,82.120613
